In [1]:
import json
import pandas as pd
departments=["nephrology department",
             "gynecology department",
            "endocrinology department", 
            "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",
             "gastroenterology department",
             "urinary surgical department",
             "hepatobiliary and pancreas surgical department",
             "hematology department"
            ]


def get_statistics(type):
    with open(f"results/analysis/{type}.json", "r") as file:
        chart = json.load(file) 
    total_cases=0
    department_wise_statistics={}
    for department in departments:
        department_wise_statistics[department]={}
        with open(f"results/{type}/{department}_{type}.json", "r") as file:
            data = json.load(file)
        all_ids=data.keys()
        total_cases+=len(all_ids)
        department_results=chart[department]
        models=list(department_results.keys())
        none_detected=[]
        all_detected=[]
        atleast_one_detected=[]
        for id  in all_ids:
            count=0
            for model in models:
                if id in department_results[model]["true"]:
                    count+=1
            if count==0:
                none_detected.append(id)
            elif count==len(models):
                all_detected.append(id)
            else:
                atleast_one_detected.append(id)
        department_wise_statistics[department]["all_detected"]=all_detected
        department_wise_statistics[department]["none_detected"]=none_detected
        department_wise_statistics[department]["atleast_one_detected"]=atleast_one_detected

    df = pd.DataFrame.from_dict(department_wise_statistics, orient='index')
    df.index.name = 'department'
    df['len_all_detected'] = df['all_detected'].apply(len)
    df['len_none_detected'] = df['none_detected'].apply(len)
    df['len_atleast_one_detected'] = df['atleast_one_detected'].apply(len)
    df.loc['Total'] = df.sum()
    # df.to_csv(f'results/analysis/department_wise_statistics_{type}.csv')
    
    filtered_df = df[['len_all_detected', 'len_none_detected','len_atleast_one_detected']]
    filtered_df.to_csv(f'results/analysis/department_wise_statistics_{type}.csv')
    return filtered_df

In [2]:
types=["single_shot","with_reasons","check_others_input"]
for type in types:
    get_statistics(type)

In [7]:
import json
import pandas as pd
departments=["nephrology department",
             "gynecology department",
            "endocrinology department", 
            "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",
             "gastroenterology department",
             "urinary surgical department",
             "hepatobiliary and pancreas surgical department",
             "hematology department"
            ]
models=["gpt-4o","llama3.1","gemma2"]

with open(f"results/analysis/with_reasons.json", "r") as file:
    first_dict = json.load(file)
with open(f"results/analysis/check_others_input.json", "r") as file:
    second_dict = json.load(file)
all_dict={}
for department in departments:
    all_dict[department]={}
    first_department=first_dict[department]
    second_department=second_dict[department]
    correct=[]
    false=[]
    all=first_department[models[0]]["true"]+first_department[models[0]]["false"]
    all.sort()
    for model in models:
        first_correct=first_department[model]["true"]
        second_correct=second_department[model]["true"]
        both_fail=0
        improve=0
        second_fail=0
        both_correct=0
        for key in all:
            if key in first_correct:
                if key in second_correct:
                    both_correct+=1
                else:
                    second_fail+=1
            elif key in second_correct:
                improve+=1
            else:
                both_fail+=1
        all_dict[department][model]={"both_fail":both_fail,"both_correct":both_correct,
                                     "improved":improve,"second_failed":second_fail}
for model in models:
    model_results = []
    for department in departments:  
        results = all_dict[department][model]
        results["department"] = department
        model_results.append(results)
    
    # Create DataFrame
    df = pd.DataFrame(model_results)
    
    # Set 'department' as the index
    df.set_index('department', inplace=True)
    
    # Add a row with the sum of all columns
    df.loc['Total'] = df.sum(numeric_only=True)  # Ensure it sums only numeric columns
    
    # Save to CSV
    df.to_csv(f"results/analysis/improvement_{model}.csv")

            

In [5]:
import json
import pandas as pd

model_dict = {}
types = ["single_shot", "with_reasons", "check_others_input"]
models = ["gpt-4o", "llama3.1", "gemma2"]

for model in models:
    temp_dict = {}
    for type in types:
        with open(f"results/analysis/{type}.json", "r") as file:
            data = json.load(file)
        
        extracted_values = {outer_key: len(inner_dict[model]["true"]) for outer_key, inner_dict in data.items()}
        temp_dict[type] = extracted_values
    
    # Create DataFrame
    df = pd.DataFrame.from_dict(temp_dict)
    
    # Add a row with the sum of all columns
    df.loc['Total'] = df.sum()
    df.index.name = 'department'
    
    # Save to CSV
    df.to_csv(f"results/analysis/{model}_performance.csv")
